In [1]:
load('Lie.sage')
load('spo.sage')
load('MatSp.sage')
load('AlgebraBSC.sage')
load('pbw.sage')

In [ ]:
#检查李超代数
spo = SpO(1,2) #spo(2,4)李超代数
mats = spo.all_matrixs
odd_mats = spo.odd_mats
N = AlgebraBSC.lie_sup_SC(mats,odd_mats)
L = AlgebraBSC(N)
v = L.basis #结构常数代数的基
n = len(N) #空间维数
syms = symbols('v',n)
odds = [syms[spo.roots.index(r)] for r in spo.odd_roots]
b = [SPBWElement(i,N,syms,odds) for i in syms]
odd_elements = [b[0](i) for i in odds]
print('开始检查')
print(check_sup_pbw(v,b,f)) 

In [4]:
#Verma模
L = SpO(1,2)
mats = L.negative_matrixs + L.h_matrixs + L.positive_matrixs
m = len(L.positive_roots)
es = Lie.symbols('e',m)
fs = Lie.symbols('f',m)
hs = Lie.symbols('h',3) #1+2
syms = fs + hs +es

roots = L.positive_roots
ind1 = [roots.index(r) for r in L.odd_roots if r in roots]
roots = L.simple_roots
ind2 = [roots.index(r) for r in L.odd_roots if r in roots]
odds = [fs[i] for i in ind1] + [hs[i] for i in ind2] + [es[i] for i in ind1]

#结构常数基
N = AlgebraBSC.lie_sup_SC(mats,L.odd_mats)
Lf = [Element(i,N,syms) for i in fs]
Le = [Element(i,N,syms) for i in es]
Lh = [Element(i,N,syms) for i in hs]
#pbw基
vf = [SPBWElement(i,N,syms,odds) for i in fs]
ve = [SPBWElement(i,N,syms,odds) for i in es]
vh = [SPBWElement(i,N,syms,odds) for i in hs]
vectors = L.vectors

In [16]:
a = (ve[1]*vf[1]*vf[1])
print(a)
print(a.to_verma(vectors[0],L,3))

{(f2^2, e2): 1, (f2, h2): 2, (f2,): -2}
{(f2,): -4}


In [17]:
l = [Le[1],Lf[1],Lh[1],Lf[1]]
for i in l:
    for j in l:
        print(i*j,end='\t')
    print()
print(L.root_on_diag(vectors[0],L.h_matrixs[1]))

0	h2	-2*e2	h2	
-h2	0	2*f2	0	
2*e2	-2*f2	0	-2*f2	
-h2	0	2*f2	0	
-1


In [3]:
class SPBWElement(PBWElement):
    '''李超代数的pbw基（修改两个函数）
    '''
    
    def __init__(self,element,N,syms,odds):
        '''初始元素为字典或U1上的符号变量'''
        assert isinstance(element,(dict,Expression)),'输入基元格式不对'
        L_dim = len(N) #李代数的维数
        self.syms = syms #李代数的符号基，用作pbw序
        if isinstance(element,Expression):
            element = Lie.coefficients(element,syms)
            element = {(syms[i],):element[i] for i in range(L_dim) if element[i]} #字典形式
        else:
            element = {key:val for key,val in zip(element.keys(),element.values()) if val}
        self.element = element #统一化字典形式，并消去零元
        self.L_dim,self.N = L_dim,N
        self.odds = odds #奇根对应的符号元
        self.keys = element.keys()
        
    def __call__(self,obj):
        '''元素转pbw类'''
        return SPBWElement(obj,self.N,self.syms,self.odds)
        
    def tuple2dict(self,element,c=1):
        '''将符号变量元组转为字典，系数默认为1'''
        if not c:return {} #零元情形
        syms = self.syms
        odds = self.odds
        n = len(element) #元素长度
        for i in range(n-1): #奇元素情形！
            if element[i] in odds and element[i]==element[i+1]:
                return {}
        N = self.N #结构常数
        coefs = [syms.index(i) for i in element] #获取对应系数
        for i in range(n-1):
            if coefs[i]>coefs[i+1]: #发现反序
                x,y = syms.index(element[i]),syms.index(element[i+1])
                e = element[:i] + (element[i+1],element[i]) + element[i+2:] #逆序处理
                seq = [(c*coef,sym) for coef,sym in zip(N[x][y],syms) if coef] #李括号项
                seq = [(coef,element[:i]+(sym,)+element[i+2:]) for coef,sym in seq]
                break
        else: #一切正序
            return {element:c}
        if element[i] in odds and element[i+1] in odds:
            c = -c
        res = self.tuple2dict(e,c)
        for coef,sym in seq:
            new = self.tuple2dict(sym,coef)
            res = self.add_dict(res,new)
        return res

In [ ]:
#调试一直报错，李超奇矩阵的次数只有2！
f = lambda a,b:a*b+(b*a if a in odd_elements and b in odd_elements else -b*a)
def check_sup_pbw(v,b,f):
    '''检验李超pbw类的正确性，v为结构常数下的代数基，b为pbw基'''
    b0 = b[0]
    for vi,bi in zip(v,b):
        for vj,bj in zip(v,b):
            bk = f(bi,bj)
            bb = b0((vi*vj).sym) 
            if bb != bk: #检查一阶情形
                #print(vi,vj,bi,bj,bb,bk)
                return False
    print('U1检验通过')
    for i,bi in enumerate(b):
        print(True,':',i)
        for bj in b:
            for bk in b: #检验结合律
                if (bi*bj)*bk != bi*(bj*bk):
                    return bi,bj,bk
    print('结合律检验通过')
    return True